<a href="https://colab.research.google.com/github/HarmanBhutani/ML_projects/blob/main/Assignment4_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



<H1 align="center"><b> Assignment 4 - Part B</b></H1>

<H1 align="center"><b> By: Harman Bhutani - 300144160 </b></H1>

<H5 align="left"><b> Overview:</b></H5>
<p align="justify">This assignment should be completed individually using either Python or R. Upon completion, present your result in one submission, including the report, answers generated and plots. Also, submit the source codes used to generate your results as a separate attachment.</p>
<H5 align="left"><b>Part B: Semantic Analysiss</b></H5>

<p align="justify"> Named Entity Recognition (NER) enables the recognition of names of persons, locations, organizations, dates in text documents. It involves two sub-tasks - identifying the boundaries of expressions (open & close brackets) and labeling the expressions with tags (e.g., PER, LOC or ORG). Using the reuters news dataset provided, complete the following:
<ol>

<li>Choose good features for encoding the problem and encode the dataset (10 points)</li>
<li>Run a classifier over the training dataset (10 points)</li>
<li>Fine tune model parameters (10 points)</li>
<li>Perform evaluation, report the accuracy, precision, recall, F-measure (10 points)</li>
<li>Return a confusion matrix (10 points)</li>
</ol></p>

In [2]:
pip install contractions

     |████████████████████████████████| 266kB 34.8MB/s 
     |████████████████████████████████| 327kB 41.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85393 sha256=02b041b7738864fa34f8bba51f7ebae24d6405579365e50e064c5d61b267d37c
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [79]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [80]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
import re
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import gensim
from gensim import corpora
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns

In [85]:
df_reut = pd.read_excel('reuters_news_headlines.xlsx', header=None)

In [86]:
df_reut.head()

,0,1,2
0,Headlines,Time,Description
1,TikTok considers London and other locations fo...,Jul 18 2020,TikTok has been in discussions with the UK gov...
2,Disney cuts ad spending on Facebook amid growi...,Jul 18 2020,Walt Disney has become the latest company to ...
3,Trail of missing Wirecard executive leads to B...,Jul 18 2020,Former Wirecard chief operating officer Jan M...
4,Twitter says attackers downloaded data from up...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...


In [87]:
df_reut[0]

0                                                Headlines
1        TikTok considers London and other locations fo...
2        Disney cuts ad spending on Facebook amid growi...
3        Trail of missing Wirecard executive leads to B...
4        Twitter says attackers downloaded data from up...
                               ...                        
32766    Malaysia says never hired British data firm at...
32767    Prosecutors search Volkswagen headquarters in ...
32768     McDonald's sets greenhouse gas reduction targets
32769    Pratt & Whitney to deliver spare A320neo engin...
32770    UK will always consider ways to improve data l...
Name: 0, Length: 32771, dtype: object

In [84]:
df_reut[0] = df_reut[0].apply(lambda x: ' '.join(x))

In [52]:
c = df_reut[0].apply(lambda x: ''.join(x))

In [88]:
nlp = en_core_web_sm.load()

In [89]:
labels = []
for text in df_reut[0]:
    en = nlp(text).ents
    labels.append([l.label_ for l in en])

In [91]:
labels[:15]

[[],
 ['GPE'],
 [],
 ['ORG', 'GPE', 'GPE'],
 ['CARDINAL'],
 ['GPE', 'NORP'],
 ['EVENT', 'WORK_OF_ART'],
 ['CARDINAL'],
 ['GPE', 'ORG'],
 ['ORG'],
 ['GPE'],
 ['ORG'],
 [],
 ['ORG', 'ORG'],
 ['ORG']]

In [92]:
label_bool = []
for idx, lab in enumerate(labels):
    if len(lab)!=0:
        label_bool.append(True)
    else:
        label_bool.append(False)

In [93]:
df_reut['labels'] = labels

In [94]:
df_reut.head()

,0,1,2,labels
0,Headlines,Time,Description,[]
1,TikTok considers London and other locations fo...,Jul 18 2020,TikTok has been in discussions with the UK gov...,[GPE]
2,Disney cuts ad spending on Facebook amid growi...,Jul 18 2020,Walt Disney has become the latest company to ...,[]
3,Trail of missing Wirecard executive leads to B...,Jul 18 2020,Former Wirecard chief operating officer Jan M...,"[ORG, GPE, GPE]"
4,Twitter says attackers downloaded data from up...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...,[CARDINAL]


1) Choose good features for encoding the problem and encode the dataset (10 points)

In [96]:
df_reut = df_reut.loc[label_bool,:]

In [97]:
mlb = MultiLabelBinarizer()

res = pd.DataFrame(mlb.fit_transform(df_reut['labels']),
                   columns=mlb.classes_,
                   index=df_reut.index)

In [98]:
res.head(10)

,CARDINAL,DATE,EVENT,FAC,GPE,LANGUAGE,LAW,LOC,MONEY,NORP,ORDINAL,ORG,PERCENT,PERSON,PRODUCT,QUANTITY,TIME,WORK_OF_ART
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
10,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [99]:
res.sum()

CARDINAL        2656
DATE            4414
EVENT             84
FAC              124
GPE            14093
LANGUAGE          11
LAW               75
LOC              964
MONEY           2670
NORP            3240
ORDINAL          463
ORG            19389
PERCENT          864
PERSON          3965
PRODUCT         1087
QUANTITY          44
TIME              80
WORK_OF_ART       68
dtype: int64

In [100]:
res = res[['ORG','GPE','DATE','CARDINAL']]

In [101]:
res.head()

,ORG,GPE,DATE,CARDINAL
1,0,1,0,0
3,1,1,0,0
4,0,0,0,1
5,0,1,0,0
6,0,0,0,0


In [141]:
corpus_value = [str (item) for item in df_reut[0]]

In [162]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,max_features=150,norm='l2', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(corpus_value).toarray()

In [171]:
features_dataf = pd.DataFrame(features)
features_dataf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.417797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.392243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.388821,0.400145,0.0,0.426731,0.0,0.0


In [173]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_dataf, res, test_size=0.30, random_state=42)

2) Fine tune model parameters (10 points)

In [174]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 200, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 51, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [175]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(features_dataf, res)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.2min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [176]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': 42,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 152}

3) Run a classifier over the training dataset (10 points)

In [177]:
rf = RandomForestClassifier(n_estimators= 152,min_samples_split= 2,min_samples_leaf= 2,max_features= 'auto',max_depth= 42,bootstrap= False)

In [178]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=42, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=152,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [179]:
predicted = rf.predict(X_test)
y_true = y_test.to_numpy()

In [180]:
class_names = ['ORG','GPE','DATE','CARDINAL']

4) Perform evaluation, report the accuracy, precision, recall, F-measure (10 points)

In [181]:
f1score = f1_score(y_true, predicted, zero_division=1,average='samples')
print('The F1 Score is',f1score)
acc = accuracy_score(y_true, predicted)
print('The Accuracy Score is',acc)
recall = recall_score(y_true, predicted, average='samples')
print('The Recall Score is',recall)
precision = precision_score(y_true, predicted, average='samples')
print('The Precision Score is',precision)

The F1 Score is 0.6761663838434626
The Accuracy Score is 0.4802023757149142
The Recall Score is 0.647968910397419
The Precision Score is 0.7411735591728992


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5) Return a confusion matrix (10 points)

In [182]:
matrix = confusion_matrix(y_true.argmax(axis=1), predicted.argmax(axis=1))

In [183]:
print(matrix)

[[5971  491   43    4]
 [1232  942   27    1]
 [ 191   26   39    1]
 [ 109   11    1    3]]
